In [9]:
%pip install openpyxl
%pip install pyinstaller
%pip install --upgrade pip

# !pyinstaller --onefile --windowed fiabilisation.ipynb

Note: you may need to restart the kernel to use updated packages.



Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re

def validate_excel(file_path, output_path):

    df = pd.read_excel(file_path)

    invalid_rows = []

#0- Constantes
    valid_domains = ["gmail.com", "hotmail.fr","hotmail.com", "yahoo.com", "yahoo.fr", "gmail.fr", "outlook.com", "icloud.com", "icloud.fr", "ucad.edu.sn","outlook.fr","cofinacorp.com","live.fr","hotmail.it","gainde2000.sn"]

    # required_packs = [
    #     "COMPTE COURANT STAFF", "EPARGNE LIBRE PARTICULIER", "PACK NDANANE","PACK DALAL" ,"PACK CLASSIC" , "PACK PRIVILEGE" ,
    #     "PACK NJEGUEMAR'LA", "PACK SOXNA'LA" ,"EPARGNE LIBRE STAFF","PACK TERANGA","EPARGNE YAKHANAAL", "EPARGNE LIBRE DIASPORA CSF"
    # ]

    for idx, row in df.iterrows():
        Matricule_Client = row.get('Matricule Client', 'Inconnu')
        Nom_Client       = row.get('Nom Client',       'Inconnu')
        Date_Ouverture_Compte       = row.get('Date Ouverture Compte',       'Inconnu')
        Agence           = row.get('Agence',           'Inconnu')
        Num_compte       = row.get('N° Compte',        'Inconnu')
        CC               = row.get('CC',               'Inconnu')
        
        errors = {
            "Matricule Client": Matricule_Client,
            "Nom Client": Nom_Client,
            "Date Ouverture Compte": Date_Ouverture_Compte ,
            "Agence":Agence,
            "N° Compte":Num_compte,
            "CC":CC,
            
            "Format du Numéro de Téléphone Invalide": [],
            "Domaine ou Format de l'Email Invalide": "",
            "Sexe ou Genre Incorrect ou Manquant pour Entreprise": "",
            "Représentant Légal Manquant": ""
        }
        country_phone_rules = {
            'Burkina Faso': {'code': '226', 'length': 8},
            'Côte d\'Ivoire': {'code': '225', 'length': 10},
            'Guinée': {'code': '224', 'length': 9},
            'Mali': {'code': '223', 'length': 8},
            'Sénégal': {'code': '221', 'length': 9},
            'Togo': {'code': '228', 'length': 8},
            'Congo': {'code': '242', 'length': 9},
            'Congo (RDC)': {'code': '243', 'length': 9},
            'Gabon': {'code': '241', 'length': 8},
            'France': {'code': '33', 'length': 9},
            'Maroc': {'code': '212', 'length': 9},
            'Espagne': {'code': '34', 'length': 9},
            'Belgique': {'code': '32', 'length': 9},
            'Suisse': {'code': '41', 'length': 9},
            'Allemagne': {'code': '49', 'length': 10},
            'Italie': {'code': '39', 'length': 10},
            'Portugal': {'code': '351', 'length': 9},
            'Pays-Bas': {'code': '31', 'length': 9},
            'États-Unis': {'code': '1', 'length': 10},
            'Canada': {'code': '1', 'length': 10},
            'Royaume-Uni': {'code': '44', 'length': 10},
            'Afrique du Sud': {'code': '27', 'length': 9},
            'Égypte': {'code': '20', 'length': 10},
            'Tunisie': {'code': '216', 'length': 8},
            'Algérie': {'code': '213', 'length': 9},
            'Ghana': {'code': '233', 'length': 9},
            'Nigéria': {'code': '234', 'length': 10},
            'Kenya': {'code': '254', 'length': 10},
            'Tanzanie': {'code': '255', 'length': 9},
            'Ouganda': {'code': '256', 'length': 9},
            'Inde': {'code': '91', 'length': 10},
            'Chine': {'code': '86', 'length': 11},
            'Japon': {'code': '81', 'length': 10},
            'Brésil': {'code': '55', 'length': 11},
            'Argentine': {'code': '54', 'length': 10},
            'Mexique': {'code': '52', 'length': 10},
            'Australie': {'code': '61', 'length': 9},
            'Nouvelle-Zélande': {'code': '64', 'length': 9},
            'Turquie': {'code': '90', 'length': 10},
            'Russie': {'code': '7', 'length': 10},
            'Arabie Saoudite': {'code': '966', 'length': 9},
            'Émirats Arabes Unis': {'code': '971', 'length': 9},
            'Qatar': {'code': '974', 'length': 8},
            'Koweït': {'code': '965', 'length': 8},
            'Bahreïn': {'code': '973', 'length': 8},
            'Oman': {'code': '968', 'length': 8}
        }


#1- Numéro de téléphone
        # Validation généralisée pour les numéros de téléphone
        phone_fields = {
            'Telephone Client': True,  # Obligatoire
            'TEL_DOM': False,          # Facultatif
            'TEL_BUR': False           # Facultatif
        }

        # Parcours de chaque champ de numéro de téléphone
        for field_name, is_required in phone_fields.items():
            phone_raw = row.get(field_name, '')
            phone_raw = str(phone_raw).strip()  # Éliminer les espaces inutiles
            phone = re.sub(r'\D', '', phone_raw.split('.')[0]) if not pd.isna(phone_raw) else ""

            if not phone:
                if is_required:
                    errors["Format du Numéro de Téléphone Invalide"] = (
                        f"{field_name} est manquant ou vide (ce champ est obligatoire)."
                    )
                continue  # Passe au champ suivant si facultatif et vide

            valid_format = False

            for country, rules in country_phone_rules.items():
                valid_number_without_code = len(phone) == rules['length']
                valid_number_with_code = (
                    phone.startswith(rules['code']) and len(phone) == rules['length'] + len(rules['code']) or
                    phone.startswith('00' + rules['code']) and len(phone) == rules['length'] + len(rules['code']) + 2 or
                    phone.startswith('+' + rules['code']) and len(phone) == rules['length'] + len(rules['code']) + 1
                )
                if valid_number_without_code or valid_number_with_code:
                    valid_format = True
                    break

            if not valid_format:
                errors["Format du Numéro de Téléphone Invalide"] = (
                    f"{field_name} '{phone_raw}' a un format invalide : indicatif et/ou taille incorrects."
                )

        # Suppression de la clé d'erreur si aucune erreur n'est présente
        if "Format du Numéro de Téléphone Invalide" in errors and not errors["Format du Numéro de Téléphone Invalide"]:
            del errors["Format du Numéro de Téléphone Invalide"]

            
#2- Adresse mail
        # Liste des extensions valides pour les emails
        valid_extensions = r'(com|org|net|edu|gov|mil|int|info|biz|fr|sn|us|uk|ca|de|es|cn|in|br|jp|au|online|tech|site|store|app|io|xyz|club|blog|bank|law|pharma|media|travel|shop|it)$'

        # Récupération et nettoyage de l'email
        email = str(row.get('Email Client', '')).strip() if not pd.isna(row.get('Email Client', '')) else ""

        # Initialisation des erreurs
        email_errors = []

        if not email:
            # Erreur si l'email est absent
            errors["Domaine ou Format de l'Email Invalide"] = "Adresse email obligatoire manquante."
        else:
            # Vérification de la présence de "@" dans l'email
            if "@" not in email:
                email_errors.append("Adresse email invalide : le symbole '@' est manquant.")
            else:
                # Séparation de la partie locale et du domaine
                local_part, domain = email.split('@', 1)

                # Vérification du domaine
                if "." not in domain:
                    email_errors.append("Adresse email invalide : le domaine est incorrect (pas de point).")
                else:
                    # Vérification de l'extension
                    extension = domain.split('.')[-1]
                    if not re.match(valid_extensions, extension):
                        email_errors.append(f"Extension du domaine invalide : .{extension} n'est pas autorisée.")
                
                # Vérification du format général de l'email
                if not re.match(r'^[\w\.-]+@[\w\.-]+\.' + valid_extensions, email):
                    email_errors.append("Format général de l'adresse email invalide.")

        # Ajout des erreurs au dictionnaire principal si elles existent
        if email_errors:
            errors["Domaine ou Format de l'Email Invalide"] = " | ".join(email_errors)

#3- Genre
        # Récupération et nettoyage des champs
        sexe = str(row.get('SEXE', '')).strip().upper() if not pd.isna(row.get('SEXE', '')) else ""
        genre_entreprise = str(row.get('Genre Pour Entreprise', '')).strip().upper() if 'Genre Pour Entreprise' in row and not pd.isna(row.get('Genre Pour Entreprise', '')) else ""
        type_pack = str(row.get('Type de Pack', '')).strip().upper() if not pd.isna(row.get('Type de Pack', '')) else ""

        sexe_normalized = ""
        genre_entreprise_normalized = ""

        # Normalisation des valeurs pour "SEXE"
        if sexe in ["F", "FEMININ", "FEMME"]:
            sexe_normalized = "F"
        elif sexe in ["M", "MASCULIN", "HOMME"]:
            sexe_normalized = "M"

        # Normalisation des valeurs pour "Genre Pour Entreprise" (si présent)
        if genre_entreprise in ["F", "FEMININ", "FEMME"]:
            genre_entreprise_normalized = "F"
        elif genre_entreprise in ["M", "MASCULIN", "HOMME"]:
            genre_entreprise_normalized = "M"

        # Validation en fonction du "Type de Pack"
        if type_pack == "Personne physique":
            if not sexe_normalized:
                errors["Sexe ou Genre Incorrect ou Manquant pour Entreprise"] = (
                    "Sexe obligatoire manquant pour le pack 'Personne physique'."
                )
            elif genre_entreprise and not genre_entreprise_normalized:
                errors["Sexe ou Genre Incorrect ou Manquant pour Entreprise"] = (
                    f"Le genre pour entreprise '{row.get('Genre Pour Entreprise', '')}' est incorrect."
                )
            elif sexe_normalized and genre_entreprise_normalized and sexe_normalized != genre_entreprise_normalized:
                errors["Sexe ou Genre Incorrect ou Manquant pour Entreprise"] = (
                    "Sexe et Genre Pour Entreprise ne correspondent pas."
                )
        else:
            # Si le type de pack n'est pas "Personne physique", on n'exige pas le sexe
            if genre_entreprise and not genre_entreprise_normalized:
                errors["Sexe ou Genre Incorrect ou Manquant pour Entreprise"] = (
                    f"Le genre pour entreprise '{row.get('Genre Pour Entreprise', '')}' est incorrect."
                )
            elif sexe_normalized and genre_entreprise_normalized and sexe_normalized != genre_entreprise_normalized:
                errors["Sexe ou Genre Incorrect ou Manquant pour Entreprise"] = (
                    "Sexe et Genre Pour Entreprise ne correspondent pas."
                )


#4- Représentant Légale
        pack_type = str(row.get('Type de Pack', '')).upper() if not pd.isna(row.get('Type de Pack', '')) else ""
        representant_legal = str(row.get('Representant Legal', '')).strip() if not pd.isna(row.get('Representant Legal', '')) else ""

        # Vérification des erreurs
        if pack_type == "Personne morale":
            # Le représentant légal est obligatoire pour les "Personne Morale"
            if not representant_legal:
                errors["Représentant Légal Manquant"] = "Représentant Légal manquant pour le pack 'Personne Morale'."
              
        if any(errors[column] for column in errors if column not in ["Matricule Client", "Nom Client", "Date Ouverture Compte", "Agence", "N° Compte", "CC"]):
            invalid_rows.append(errors)

#5- Fichier retourné
    if invalid_rows:
        invalid_df = pd.DataFrame(invalid_rows)
        invalid_df.to_excel(output_path, index=False)
        return invalid_df
    else:
        return pd.DataFrame()  # Retourner un DataFrame vide


# Utilisation
file_path = "C:/Users/djibril.marwan/Desktop/Fiabilisation SN test.xlsx"
output_path = "C:/Users/djibril.marwan/Desktop/invalid_rows.xlsx"
invalid_data = validate_excel(file_path, output_path)

# Affiche le nombre de lignes avec erreurs et le fichier Excel généré
if isinstance(invalid_data, pd.DataFrame):
    print(f"Nombre de lignes avec erreurs: {invalid_data.shape[0]}")
    print("Le fichier Excel contenant les lignes invalides par condition a été généré.")
else:
    print(invalid_data)  # Affiche "Toutes les lignes répondent aux critères de validation" si tout est correct

print(invalid_data)

Nombre de lignes avec erreurs: 55
Le fichier Excel contenant les lignes invalides par condition a été généré.
    Matricule Client                   Nom Client Date Ouverture Compte  \
0           17034911               ELHADJI MAMATH            2024-05-14   
1           17081611                     SEYNABOU            2024-05-23   
2           17101211                          NaN            2024-05-28   
3           17066711  MANUELLE FORTUNE JUDICAELLE            2024-05-21   
4           17096611                          NaN            2024-05-27   
5           16994511                      KHALIFA            2024-05-03   
6           17013311                          NaN            2024-05-08   
7           17020011                    SECK YORO            2024-05-10   
8           17038411         NDEYE AMINATA D ANTA            2024-05-15   
9           17075111                KHOUDIA NIANG            2024-05-22   
10          17051811                        MASSE            2024

In [11]:
# !python -m jupyter nbconvert --to script fiabilisation.ipynb